<a href="https://colab.research.google.com/github/khataei/PE-classification-DeepLearning/blob/master/Tunned-Talos-1-CNN-activity-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Talos Tuner for CNN Activity Classifier

In this notebook, we use SKlearn  to tune a CNN neural net to classify PE activity.

#### Load dependencies

In [1]:
import os  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.layers import AveragePooling1D, LeakyReLU , MaxPool1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import roc_auc_score, roc_curve 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
import tensorflow as tf
print("# GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tensorflow.__version__

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4021926830220582436
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7005546176762618463
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4022219571
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8187960795673950703
physical_device_desc: "device: 0, name: Quadro P2000, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16589542880459548679
physical_device_desc: "device: XLA_GPU device"
]
# GPUs Available:  1


'2.3.1'

#### Set hyperparameters

In [2]:
# output directory name:
output_dir = 'model_output/tunecnn1'
input_dir =  'Z:/Research/dfuller/Walkabilly/studies/smarphone_accel/data/Ethica_Jaeger_Merged/pocket/'
input_file_name = 'pocket-NN-data.npz'

# from the data preparation section we have:
window_size_second = 3
frequency = 30
lenght_of_each_seq = window_size_second * frequency


In [3]:
# sklearn hyperparams
params = {
    'n_conv_1':[256, 512, 768], # filters, a.k.a. kernels
    'k_conv_1':[3, 5], # kernel length
    'n_conv_2':[256, 512, 768], # filters, a.k.a. kernels
    'k_conv_2':[3, 5], # kernel length
    'n_conv_3':[256, 512, 768], # filters, a.k.a. kernels
    'k_conv_3':[3, 5], # kernel length
    'maxpooling_pool_size':[2, 4],
    'avepooling_pool_size':[2, 4],
    'n_dense_1':[256, 512],
    'dropout_1':[0.2, 0.4],
    'n_dense_2':[256, 512],
    'dropout_2':[0.2, 0.4],
    'activation':['elu', 'relu', LeakyReLU()]
}

# training:
n_tune_iter = 40
cv = 5
epochs = 60
batch_size = 256


#### Load data

##### For this notebook we use the acceleration data gathered from the pocket location. It was prepared in the DataPrep-Deep notebook

In [4]:
# read the raw file and get the keys:
raw_data = np.load(file=input_dir+input_file_name,allow_pickle=True)
for k in raw_data.keys():
    print(k)

acceleration_data
metadata
labels


In [5]:
# import the data

accel_array = raw_data['acceleration_data']
meta_array = raw_data['metadata']
labels_array = raw_data['labels']
input_shape = list(accel_array.shape)


#### Preprocess data

#### Convert the  labels to integer.
In the raw data format of the labels is String and there are 6 classes. 'Lying', 'Sitting', 'Self Pace walk', 'Running 3 METs',
       'Running 5 METs', 'Running 7 METs' <br>




In [6]:
n_class = len(np.unique(labels_array))
class_list, labels_array_int = np.unique(labels_array,return_inverse=True)

In [7]:
y = to_categorical(labels_array_int, num_classes=n_class)


### Splitting and shuffeling the data

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(
     accel_array, y, test_size=0.1, random_state=65)


#### Design neural network architecture

In [9]:
params

{'n_conv_1': [256, 512, 768],
 'k_conv_1': [3, 5],
 'n_conv_2': [256, 512, 768],
 'k_conv_2': [3, 5],
 'n_conv_3': [256, 512, 768],
 'k_conv_3': [3, 5],
 'maxpooling_pool_size': [2, 4],
 'avepooling_pool_size': [2, 4],
 'n_dense_1': [256, 512],
 'dropout_1': [0.2, 0.4],
 'n_dense_2': [256, 512],
 'dropout_2': [0.2, 0.4],
 'activation': ['elu',
  'relu',
  <tensorflow.python.keras.layers.advanced_activations.LeakyReLU at 0x238cb4127c8>]}

In [10]:
def create_model(n_conv_1=256, k_conv_1=3, n_conv_2=256, k_conv_2=3, n_conv_3=256, k_conv_3=3,
                 maxpooling_pool_size = 2, avepooling_pool_size = 2, n_dense_1=256, dropout_1=0.2,
                 n_dense_2=256, dropout_2=0.2, activation= 'elu'
                ):
    model = Sequential()
    model.add(Conv1D(n_conv_1, k_conv_1, activation=activation, input_shape=input_shape[1:]))
    model.add(MaxPool1D(pool_size = maxpooling_pool_size))
    model.add(Conv1D(n_conv_2, k_conv_2, activation=activation))
    model.add(AveragePooling1D(pool_size = avepooling_pool_size))
    model.add(Conv1D(n_conv_3, k_conv_3, activation=activation))
    # model.add(GlobalMaxPooling1D())
    model.add(GlobalAveragePooling1D())
    model.add(Dense(n_dense_1, activation=activation))
    model.add(Dropout(dropout_1))
    model.add(Dense(n_dense_2, activation=activation))
    model.add(Dropout(dropout_2))
    model.add(Dense(n_class, activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

    return model

In [11]:
model_default = create_model()
model_default.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 88, 256)           2560      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 44, 256)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 42, 256)           196864    
_________________________________________________________________
average_pooling1d (AveragePo (None, 21, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 19, 256)           196864    
_________________________________________________________________
global_average_pooling1d (Gl (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               6

In [12]:
model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size)

In [ ]:
%%time
rscv = RandomizedSearchCV(model, param_distributions=params, cv=cv, n_iter=n_tune_iter)
rscv_results = rscv.fit(X_train,y_train)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 86, 256)           4096      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 43, 256)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 39, 256)           327936    
_________________________________________________________________
average_pooling1d_1 (Average (None, 9, 256)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 7, 256)            196864    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)              

183/183 [==============================] - 4s 23ms/step - loss: 1.1475 - accuracy: 0.5099
Epoch 2/60
183/183 [==============================] - 4s 23ms/step - loss: 0.9150 - accuracy: 0.6282
Epoch 3/60
183/183 [==============================] - 4s 23ms/step - loss: 0.7201 - accuracy: 0.7125
Epoch 4/60
183/183 [==============================] - 4s 23ms/step - loss: 0.6469 - accuracy: 0.7431
Epoch 5/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5674 - accuracy: 0.7723
Epoch 6/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5395 - accuracy: 0.7817
Epoch 7/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5123 - accuracy: 0.7954
Epoch 8/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4835 - accuracy: 0.8067
Epoch 9/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4631 - accuracy: 0.8169
Epoch 10/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4578 - accuracy: 0.8230


183/183 [==============================] - 4s 23ms/step - loss: 0.8908 - accuracy: 0.6385
Epoch 3/60
183/183 [==============================] - 4s 23ms/step - loss: 0.7306 - accuracy: 0.7085
Epoch 4/60
183/183 [==============================] - 4s 23ms/step - loss: 0.6747 - accuracy: 0.7361
Epoch 5/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5663 - accuracy: 0.7700
Epoch 6/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5290 - accuracy: 0.7841
Epoch 7/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5249 - accuracy: 0.7907
Epoch 8/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4785 - accuracy: 0.8066
Epoch 9/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4723 - accuracy: 0.8110
Epoch 10/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4287 - accuracy: 0.8305
Epoch 11/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4289 - accuracy: 0.8311

183/183 [==============================] - 4s 23ms/step - loss: 0.7141 - accuracy: 0.7159
Epoch 4/60
183/183 [==============================] - 4s 23ms/step - loss: 0.6297 - accuracy: 0.7477
Epoch 5/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5789 - accuracy: 0.7703
Epoch 6/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5422 - accuracy: 0.7844
Epoch 7/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5490 - accuracy: 0.7863
Epoch 8/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4725 - accuracy: 0.8131
Epoch 9/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4758 - accuracy: 0.8136
Epoch 10/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4303 - accuracy: 0.8316
Epoch 11/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4177 - accuracy: 0.8369
Epoch 12/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4072 - accuracy: 0.839

183/183 [==============================] - 4s 23ms/step - loss: 0.6450 - accuracy: 0.7442
Epoch 5/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5817 - accuracy: 0.7654
Epoch 6/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5326 - accuracy: 0.7856
Epoch 7/60
183/183 [==============================] - 4s 23ms/step - loss: 0.5123 - accuracy: 0.7959
Epoch 8/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4768 - accuracy: 0.8088
Epoch 9/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4619 - accuracy: 0.8199
Epoch 10/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4611 - accuracy: 0.8206
Epoch 11/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4261 - accuracy: 0.8338
Epoch 12/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4101 - accuracy: 0.8399
Epoch 13/60
183/183 [==============================] - 4s 23ms/step - loss: 0.4026 - accuracy: 0.84

...

183/183 [==============================] - 14s 74ms/step - loss: 0.6118 - accuracy: 0.7562
Epoch 6/60
183/183 [==============================] - 14s 74ms/step - loss: 0.5710 - accuracy: 0.7710
Epoch 7/60
183/183 [==============================] - 14s 74ms/step - loss: 0.5279 - accuracy: 0.7883
Epoch 8/60
183/183 [==============================] - 14s 74ms/step - loss: 0.6694 - accuracy: 0.7536
Epoch 9/60
183/183 [==============================] - 14s 74ms/step - loss: 0.4922 - accuracy: 0.8053
Epoch 10/60
183/183 [==============================] - 14s 74ms/step - loss: 0.4512 - accuracy: 0.8211
Epoch 11/60
183/183 [==============================] - 14s 74ms/step - loss: 0.4379 - accuracy: 0.8258
Epoch 12/60
183/183 [==============================] - 14s 74ms/step - loss: 2.2267 - accuracy: 0.7352
Epoch 13/60
183/183 [==============================] - 14s 74ms/step - loss: 0.7937 - accuracy: 0.6888
Epoch 14/60
183/183 [==============================] - 14s 74ms/step - loss: 0.6108 - acc

183/183 [==============================] - 14s 74ms/step - loss: 0.7995 - accuracy: 0.6780
Epoch 4/60
183/183 [==============================] - 14s 74ms/step - loss: 0.6631 - accuracy: 0.7346
Epoch 5/60
183/183 [==============================] - 14s 74ms/step - loss: 0.7424 - accuracy: 0.7153
Epoch 6/60
183/183 [==============================] - 14s 74ms/step - loss: 0.5688 - accuracy: 0.7724
Epoch 7/60
183/183 [==============================] - 14s 74ms/step - loss: 0.5397 - accuracy: 0.7853
Epoch 8/60
183/183 [==============================] - 14s 74ms/step - loss: 0.5005 - accuracy: 0.8013
Epoch 9/60
183/183 [==============================] - 14s 74ms/step - loss: 0.4793 - accuracy: 0.8093
Epoch 10/60
183/183 [==============================] - 14s 74ms/step - loss: 0.4658 - accuracy: 0.8141
Epoch 11/60
183/183 [==============================] - 14s 74ms/step - loss: 1.6707 - accuracy: 0.6320
Epoch 12/60
183/183 [==============================] - 14s 74ms/step - loss: 0.6346 - accur

183/183 [==============================] - 14s 74ms/step - loss: 1.2008 - accuracy: 0.4878
Epoch 2/60
183/183 [==============================] - 14s 74ms/step - loss: 0.9713 - accuracy: 0.5972
Epoch 3/60
183/183 [==============================] - 14s 74ms/step - loss: 0.7869 - accuracy: 0.6824
Epoch 4/60
183/183 [==============================] - 14s 74ms/step - loss: 0.6890 - accuracy: 0.7248
Epoch 5/60
183/183 [==============================] - 14s 74ms/step - loss: 0.6040 - accuracy: 0.7587
Epoch 6/60
183/183 [==============================] - 14s 74ms/step - loss: 0.5674 - accuracy: 0.7717
Epoch 7/60
183/183 [==============================] - 14s 74ms/step - loss: 0.5296 - accuracy: 0.7879
Epoch 8/60
183/183 [==============================] - 14s 74ms/step - loss: 0.4957 - accuracy: 0.8015
Epoch 9/60
183/183 [==============================] - 14s 74ms/step - loss: 0.5296 - accuracy: 0.7932
Epoch 10/60
183/183 [==============================] - 14s 74ms/step - loss: 0.4434 - accurac

183/183 [==============================] - 14s 78ms/step - loss: 1.1815 - accuracy: 0.4918
Epoch 2/60
183/183 [==============================] - 14s 75ms/step - loss: 0.9475 - accuracy: 0.6058
Epoch 3/60
183/183 [==============================] - 13s 74ms/step - loss: 0.7751 - accuracy: 0.6885
Epoch 4/60
183/183 [==============================] - 14s 74ms/step - loss: 0.6600 - accuracy: 0.7374
Epoch 5/60
183/183 [==============================] - 14s 75ms/step - loss: 0.5845 - accuracy: 0.7654
Epoch 6/60
183/183 [==============================] - 14s 74ms/step - loss: 0.5507 - accuracy: 0.7810
Epoch 7/60
183/183 [==============================] - 13s 73ms/step - loss: 0.6797 - accuracy: 0.7461
Epoch 8/60
183/183 [==============================] - 13s 73ms/step - loss: 0.4966 - accuracy: 0.8042
Epoch 9/60
183/183 [==============================] - 13s 73ms/step - loss: 0.4779 - accuracy: 0.8128
Epoch 10/60
183/183 [==============================] - 13s 73ms/step - loss: 0.4796 - accurac

183/183 [==============================] - 13s 73ms/step - loss: 1.1836 - accuracy: 0.4945
Epoch 2/60
183/183 [==============================] - 13s 73ms/step - loss: 0.9779 - accuracy: 0.5931
Epoch 3/60
183/183 [==============================] - 13s 72ms/step - loss: 0.7908 - accuracy: 0.6812
Epoch 4/60
183/183 [==============================] - 13s 72ms/step - loss: 0.7701 - accuracy: 0.6973
Epoch 5/60
183/183 [==============================] - 13s 72ms/step - loss: 0.6190 - accuracy: 0.7510
Epoch 6/60
183/183 [==============================] - 13s 73ms/step - loss: 0.5819 - accuracy: 0.7662
Epoch 7/60
183/183 [==============================] - 13s 73ms/step - loss: 0.5307 - accuracy: 0.7872
Epoch 8/60
183/183 [==============================] - 13s 73ms/step - loss: 0.5126 - accuracy: 0.7949
Epoch 9/60
183/183 [==============================] - 13s 74ms/step - loss: 0.5252 - accuracy: 0.7943
Epoch 10/60
183/183 [==============================] - 14s 75ms/step - loss: 0.4823 - accurac

In [ ]:
print('Best score is: {} using {}'.format(rscv_results.best_score_,
rscv_results.best_params_))

In [ ]:
best_model = rscv_results.best_estimator_
best_param = rscv_results.best_params_

### Test the best model based on the validation data

In [ ]:
y_hat = best_model.predict(X_valid)

In [ ]:
y_hat = to_categorical(y_hat)
y_hat[0]

In [ ]:
y_valid[0]

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
from sklearn.metrics import  accuracy_score as score
y_pred_classes = y_hat.round()
print(y_valid[0], y_hat[0], y_pred_classes[0])
acc = score(y_valid, y_pred_classes) * 100
acc

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat)*100.0

In [ ]:
"{:0.2f}".format(pct_auc)

In [ ]:
float_y_hat = []
for y in y_hat:
    float_y_hat.append(y[0:6].round(3))

In [ ]:
ydf = pd.DataFrame(list(zip(float_y_hat, y_valid)), columns=['y_hat', 'y'])

In [ ]:
ydf.head(10)

In [ ]:
results_df = pd.DataFrame(rscv_results.cv_results_['params'])
results_df['mean'] = rscv_results.cv_results_['mean_test_score']
results_df['std'] = rscv_results.cv_results_['std_test_score']
results_df.sort_values('mean', ascending=False, ignore_index=False)

## Creating best model from the best param and train it for 60 epochs

In [ ]:
list(best_param.items())

In [ ]:
n_dense_2= 256
n_dense_1= 512
n_conv_3= 256
n_conv_2= 768
n_conv_1= 768
maxpooling_pool_size= 2
k_conv_3= 3
k_conv_2= 5
k_conv_1= 5
dropout_2= 0.4
dropout_1= 0.4
avepooling_pool_size= 2
activation= 'relu'
model = Sequential()
model.add(Conv1D(n_conv_1, k_conv_1, activation=activation, input_shape=input_shape[1:]))
model.add(MaxPool1D(pool_size = maxpooling_pool_size))
model.add(Conv1D(n_conv_2, k_conv_2, activation=activation))
model.add(AveragePooling1D(pool_size = avepooling_pool_size))
model.add(Conv1D(n_conv_3, k_conv_3, activation=activation))
# model.add(GlobalMaxPooling1D())
model.add(GlobalAveragePooling1D())
model.add(Dense(n_dense_1, activation=activation))
model.add(Dropout(dropout_1))
model.add(Dense(n_dense_2, activation=activation))
model.add(Dropout(dropout_2))
model.add(Dense(n_class, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])


In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
modelcheckpoint = ModelCheckpoint(filepath=output_dir+
                                  "/weights.{epoch:02d}.hdf5")

In [ ]:
model.fit(X_train, y_train, 
         batch_size=batch_size, epochs=epochs, verbose=1, 
         validation_data=(X_valid, y_valid), 
         callbacks=[modelcheckpoint])

In [ ]:
model.load_weights(output_dir+"/weights.49.hdf5") # 94.22 val accuracy

In [ ]:
y_hat = model.predict(x_valid)

In [ ]:
len(y_hat)

In [ ]:
y_hat[0]

In [ ]:
y_valid[0]

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat)*100.0

In [ ]:
"{:0.2f}".format(pct_auc)

In [ ]:
float_y_hat = []
for y in y_hat:
    float_y_hat.append(y[0:6].round(3))

In [ ]:
ydf = pd.DataFrame(list(zip(float_y_hat, y_valid)), columns=['y_hat', 'y'])

In [ ]:
ydf.head(10)